In [1]:
import numpy as np
import pandas as pd

import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix

import io


from warnings import filterwarnings
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']
colors_red = ["#331313", "#582626", '#9E1717', '#D35151', '#E9B4B4']
colors_green = ['#01411C','#4B6F44','#4F7942','#74C365','#D0F0C0']


labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']
X_train = []
y_train = []
image_size = 150


In [2]:
os.chdir("C:\\Users\\Shrey\\MinorProjectDB")


for i in labels:
    folderPath = os.path.join('C:\\Users\\Shrey\\MinorProjectDB','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join('C:\\Users\\Shrey\\MinorProjectDB','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
        

100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:00<00:00, 88.32it/s]


In [3]:
X_train = np.array(X_train)
y_train = np.array(y_train)

k=0
X_train, y_train = shuffle(X_train,y_train, random_state=101)
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)

y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

In [4]:
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

model = effnet.output
model = tf.keras.layers.GlobalAveragePooling2D()(model)
model = tf.keras.layers.Dropout(rate=0.5)(model)
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

model.summary()


model.compile(loss='categorical_crossentropy',optimizer = 'Adam', metrics= ['accuracy'])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 150, 150, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 150, 150, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 151, 151, 3)  0           normalization[0][0]              
______________________________________________________________________________________________

In [5]:
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [6]:
history = model.fit(X_train,y_train,validation_split=0.1, epochs =8, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])

C:\Users\Shrey\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/8
83/83 [==============================] - 216s 3s/step - loss: 0.4949 - accuracy: 0.8195 - val_loss: 0.3881 - val_accuracy: 0.8503

Epoch 00001: val_accuracy improved from -inf to 0.85034, saving model to effnet.h5
Epoch 2/8
83/83 [==============================] - 207s 2s/step - loss: 0.1649 - accuracy: 0.9440 - val_loss: 0.3444 - val_accuracy: 0.8980

Epoch 00002: val_accuracy improved from 0.85034 to 0.89796, saving model to effnet.h5
Epoch 3/8
83/83 [==============================] - 212s 3s/step - loss: 0.1321 - accuracy: 0.9576 - val_loss: 0.7185 - val_accuracy: 0.7959

Epoch 00003: val_accuracy did not improve from 0.89796
Epoch 4/8
83/83 [==============================] - 217s 3s/step - loss: 0.1331 - accuracy: 0.9531 - val_loss: 0.2201 - val_accuracy: 0.9150

Epoch 00004: val_accuracy improved from 0.89796 to 0.91497, saving model to effnet.h5
Epoch 5/8
83/83 [==============================] - 219s 3s/step - loss: 0.0760 - accuracy: 0.9758 - val_loss: 0.1742 - val_acc

In [7]:
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

In [8]:
print(classification_report(y_test_new,pred))

              precision    recall  f1-score   support

           0       0.93      0.96      0.94        93
           1       0.94      0.94      0.94        51
           2       0.99      0.93      0.96        96
           3       0.97      1.00      0.98        87

    accuracy                           0.96       327
   macro avg       0.96      0.96      0.96       327
weighted avg       0.96      0.96      0.96       327

